# Setup

In [1]:
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
import wandb
from utilities import *
from config import *
from dataloading import *
from tqdm import tqdm
from transformer import *
import os

Loading data...


Loading data: 100%|██████████| 100000/100000 [00:00<00:00, 182848.77it/s]


In [2]:
for x in train_dataloader:
    data, train = x
    break

In [11]:
data[:10]

tensor([[10,  5,  5,  3,  6,  5, 16,  3,  2,  1, 17],
        [ 5,  9, 14,  1,  2,  2, 16,  2, 17, 17, 17],
        [ 2,  9, 14,  6,  1,  8, 16,  0,  2,  3, 17],
        [ 1, 13,  4,  6,  9,  4, 16,  1,  3,  2, 17],
        [14,  2, 13,  4,  1,  6, 16,  3,  0,  2, 17],
        [12, 13, 10, 11,  0,  1, 16,  0, 17, 17, 17],
        [13, 14,  3, 12,  2,  5, 16,  1,  3,  0, 17],
        [13, 12,  3,  6,  8,  2, 16,  0, 17, 17, 17],
        [10, 12, 14, 13,  2, 13, 16,  0,  1,  3, 17],
        [ 4,  7,  9,  5,  3,  5, 16,  0,  2, 17, 17]])

In [12]:
train[:10]

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [2]:
model = BigramLanguageModel()

In [4]:
torch.no_grad()
model.eval()

BigramLanguageModel(
  (token_embedding_table): Embedding(18, 384)
  (position_embedding): Embedding(11, 384)
  (sa_heads): MultiHeadAttention(
    (heads): ModuleList(
      (0-5): 6 x Head(
        (key): Linear(in_features=384, out_features=64, bias=False)
        (query): Linear(in_features=384, out_features=64, bias=False)
        (value): Linear(in_features=384, out_features=64, bias=False)
        (dropout): Dropout(p=0, inplace=False)
      )
    )
    (proj): Linear(in_features=384, out_features=384, bias=True)
    (dropout): Dropout(p=0, inplace=False)
  )
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-5): 6 x Head(
            (key): Linear(in_features=384, out_features=64, bias=False)
            (query): Linear(in_features=384, out_features=64, bias=False)
            (value): Linear(in_features=384, out_features=64, bias=False)
            (dropout): Dropout(p=0, inplace=False)
          )
        )
      

In [6]:
# calculate validation stats
criterion = nn.CrossEntropyLoss()

total_accuracy = 0.0
total_loss = 0.0

num_batches = 0

print("Evaluating...")
for inputs, targets in tqdm(val_dataloader):
    outputs = model(inputs)

    # calculate the val accuracy
    accuracy = calculate_accuracy(outputs, targets)
    total_accuracy += accuracy

    # Calculate the val loss
    loss = criterion(outputs, targets)
    total_loss += loss.item()
    num_batches += 1

average_accuracy = total_accuracy / num_batches
val_loss = total_loss / num_batches

Evaluating...


100%|██████████| 6250/6250 [04:00<00:00, 26.02it/s]


In [7]:
val_loss

2.890560711326599

In [8]:
vocab_size

18

In [19]:
output = model(data)

In [20]:
train

tensor([1, 0, 2, 0, 2, 2, 0, 2, 3, 2, 0, 0, 3, 1, 0, 1, 0, 1, 0, 2, 1, 3, 3, 1,
        2, 2, 0, 0, 2, 0, 2, 0, 2, 3, 1, 2, 3, 2, 2, 3, 0, 2, 0, 2, 2, 2, 3, 0,
        0, 0, 2, 2, 2, 3, 1, 0, 0, 0, 2, 2, 3, 0, 3, 3])

In [21]:
train.shape

torch.Size([64])

In [24]:
output.shape

torch.Size([64, 18])

tensor(0.0156)

In [15]:
output[1]

tensor([ 0.1212,  0.0911,  0.2244,  0.0481,  0.0014,  0.4452, -0.3902, -0.2757,
         0.2142,  0.2296,  0.5740,  1.2472,  0.7923,  0.0781,  0.3573, -0.1222,
        -0.6712,  0.9082], grad_fn=<SelectBackward0>)

In [30]:
model.get_device()

AttributeError: 'BigramLanguageModel' object has no attribute 'get_device'

In [31]:
dev

NameError: name 'dev' is not defined

In [4]:
if cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"

In [15]:
convert_to_transposition(16)

(4, 0)

In [3]:
model.generate([1,6,11,1,6,11])

[17, 17, 17, 17]

In [5]:
prediction_tensor = torch.zeros(block_size, dtype=int).to(dev)

In [36]:
sequence = [1,2,3,1,2,3]

In [37]:
prediction_tensor[:len(sequence)] = torch.tensor(sequence, dtype=int).to(dev)

In [38]:
prediction_tensor

tensor([1, 2, 3, 1, 2, 3, 0, 0, 0, 0, 0])

In [39]:
input_tensor = torch.ones(block_size, dtype=int).to(dev)

In [41]:
input_tensor *= TO_PREDICT_TOKEN

In [42]:
input_tensor

tensor([17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17])